<a href="https://colab.research.google.com/github/ngntrgduc/learning-python/blob/master/chainmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ChainMap: Manage Multiple Contexts Effectively

Lookups search the underlying mappings successively until a key is found. In contrast, writes, updates, and deletions only operate on the first mapping.

Purpose: multi-layered configuration overrides, and variable stack/scope emulation.

The trade-off with ChainMap is that lookups might be slightly slower because it has to check multiple dictionaries in order

In [1]:
from collections import ChainMap

In [2]:
user_preferences = {
    'theme': 'Dark',
    'auto_save': True,
}

default_preferences = {
    'theme': 'Light',
    'language': 'English',
    'font_size': 14,
    'auto_save': False,
}

In [3]:
user_preferences | default_preferences

{'theme': 'Light', 'auto_save': False, 'language': 'English', 'font_size': 14}

In [4]:
{**default_preferences, **user_preferences} # create new dict

{'theme': 'Dark', 'language': 'English', 'font_size': 14, 'auto_save': True}

Merging dictionaries creates a snapshot copy. Any changes to the original dictionaries do not affect the merged dictionary.

In [5]:
cm = ChainMap(user_preferences, default_preferences)
print(cm)

ChainMap({'theme': 'Dark', 'auto_save': True}, {'theme': 'Light', 'language': 'English', 'font_size': 14, 'auto_save': False})


### Attributes

The underlying mappings are stored in a list. That list is public and can be accessed or updated using the maps attribute.

In [6]:
cm.maps

[{'theme': 'Dark', 'auto_save': True},
 {'theme': 'Light',
  'language': 'English',
  'font_size': 14,
  'auto_save': False}]

In [7]:
cm.parents

ChainMap({'theme': 'Light', 'language': 'English', 'font_size': 14, 'auto_save': False})

If one of the underlying mappings gets updated, those changes will be reflected in ChainMap

In [8]:
user_preferences['font_size'] = 16
print(f'{user_preferences = }')
print(f'{cm = }')

user_preferences = {'theme': 'Dark', 'auto_save': True, 'font_size': 16}
cm = ChainMap({'theme': 'Dark', 'auto_save': True, 'font_size': 16}, {'theme': 'Light', 'language': 'English', 'font_size': 14, 'auto_save': False})


In [9]:
def test(key, dictionary, chainmap):
    """Trying to get key in both dictionary and ChainMap"""
    try:
        print(f"dictionary['{key}'] -> {dictionary[key]}")
    except KeyError:
        print(f"Could not get '{key}' in dictionary")

    try:
        print(f"ChainMap['{key}'] -> {chainmap[key]}")
    except KeyError:
        print(f"Could not get '{key}' in ChainMap")


In [10]:
test('theme', user_preferences, cm)

dictionary['theme'] -> Dark
ChainMap['theme'] -> Dark


In [11]:
test('language', user_preferences, cm)

Could not get 'language' in dictionary
ChainMap['language'] -> English


In [12]:
test('unknow_key', user_preferences, cm)

Could not get 'unknow_key' in dictionary
Could not get 'unknow_key' in ChainMap


### Shielding Other Dictionaries

useful in scenarios where you want to allow updates while preserving the integrity of the original dictionaries.

In [13]:
new_cm = cm.new_child({'font_size': 10, 'theme': 'Catppuccin'})
new_cm

ChainMap({'font_size': 10, 'theme': 'Catppuccin'}, {'theme': 'Dark', 'auto_save': True, 'font_size': 16}, {'theme': 'Light', 'language': 'English', 'font_size': 14, 'auto_save': False})

In [14]:
new_cm['rulers'] = 80
new_cm

ChainMap({'font_size': 10, 'theme': 'Catppuccin', 'rulers': 80}, {'theme': 'Dark', 'auto_save': True, 'font_size': 16}, {'theme': 'Light', 'language': 'English', 'font_size': 14, 'auto_save': False})